In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import transformers
from transformers import LayoutLMv2Processor, LayoutLMForTokenClassification, AdamW, BertForTokenClassification,\
    BertTokenizer, LayoutLMTokenizer, AutoTokenizer, AutoModelForMaskedLM, pipeline, DataCollatorForLanguageModeling
from datasets import load_dataset
import torch
from torch.nn.functional import cross_entropy
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import logging
from torch.utils.data import DataLoader, Dataset
from torcheval.metrics.functional import multiclass_f1_score, multiclass_confusion_matrix
from copy import deepcopy, copy
import seaborn as sns
import pandas as pd
from huggingface_hub import notebook_login

import os 
while 'notebooks' in os.getcwd():
    os.chdir("..")

import re
from typing import List, Dict
    
from src.preprocessing.make_dataset import ImageLayoutDataset, PatternExploitingDataset, SplitWordsDataset
from src.model.mlm.trainer import MLMTrainer
import warnings
warnings.filterwarnings("ignore")

# Iterative PET

In [3]:
!mkdir logs
model_name = "iPET"
dataset_name = "FUNSD"
device = "cuda" if torch.cuda.is_available() else "cpu"
logging.basicConfig(filename=f'logs/{model_name}_{dataset_name}.log', encoding='utf-8', level= logging.INFO)

mkdir: cannot create directory ‘logs’: File exists


## Logging in to huggingface hub

In [4]:
!pip install ipywidgets


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
notebook_login()

## Importing models and data

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [6]:
dataset = load_dataset("nielsr/funsd")

In [7]:
def pattern0(
    phrase : str,
    word : str,
    tokenizer
):
    return f"In the phrase '{phrase}', the {word} is in the header, in a question, or in an answer? {tokenizer.mask_token}"

def pattern1(
    phrase : str, 
    word : str,
    tokenizer
):
    return f"In the phrase '{phrase}', where is the {word} is situated at? {tokenizer.mask_token}"

def pattern3(
    phrase : str,
    word : str, 
    tokenizer
):
    return f"Question: In the phrase '{phrase}', "

In [8]:
pattern_list = [pattern0, pattern1,]

In [46]:
train_datas = [
    SplitWordsDataset(
        dataset['train'],
        tokenizer,
        pattern
    )

    for pattern in pattern_list
]

100%|██████████| 149/149 [00:00<00:00, 2508.46it/s]


In [10]:
verbalizer = {
    "none": 0,
    "question" : 1,
    "answer": 2,
    "header" : 3,
}

## Train all models with patterns

In [16]:
device = "cuda"
models = [
    AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased").to(device)
    for i in range(len(pattern_list))
]

trainers = [
    MLMTrainer(models[i], tokenizer, verbalizer )
    for i in range(len(pattern_list))
]


In [17]:
for idx, trainer in enumerate(trainers):
    logging.info(f"Training the {idx}th model")
    train_data = train_datas[idx]

    trainer.compile(
        train_data,
        n_shots=10,

    )
    trainer.best_model.push_to_hub(f"peulsilva/ipet-pattern{idx}-10-shots")
    

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:53<00:00,  5.38s/it]


pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

100%|██████████| 10/10 [00:53<00:00,  5.37s/it]


pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

## Implementing iPET

In [11]:
models = [[
    AutoModelForMaskedLM.from_pretrained(f"peulsilva/ipet-pattern{i}-10-shots").to(device)
    for i in range(2)
]]

trainers = [
    MLMTrainer(models[0][i], tokenizer, verbalizer )
    for i in range(len(pattern_list))
]

In [59]:
T_ij[-1][]

In [72]:
n_models = 2
n_patterns = 2
T_ij = [deepcopy(train_datas)]
for j in range(n_models):
    logging.info(f"Evaluating dataset with {(j+1)*5} shots")
    # evaluate
    new_T = []
    for i in range(n_patterns):
        Lambda = np.random.randint(1,2)
        # Lambda*(n-1)
        trainer = MLMTrainer(models[-1][1-i], tokenizer, verbalizer)
        generated_labels ,_, _  = trainer.evaluate(
            T_ij[-1][i], 
            models[-1][1-i],
            n_shots = 5*(1+j),
            return_generated_dataset=True
        )

        logging.info("Ended generating labels")
        T_n = deepcopy(train_datas)
        for j_i in range(len(T_n[i])):
            for k in range(len(T_n[i][j_i])):
                if j_i < len(generated_labels):
                    T_n[i][j_i][k]['label'] = generated_labels[j_i][k]
        new_T.append(T_n[i])

    T_ij.append(new_T)

    logging.info(f"Training models with {(j+1)*5} shots")
    # train
    models.append([
        AutoModelForMaskedLM.from_pretrained(f"peulsilva/ipet-pattern{i}-10-shots").to(device)
        for i in range(2)
    ])
    for i in range(n_patterns):
        trainer = MLMTrainer(
            models[-1][i],
            tokenizer,
            verbalizer
        )

        trainer.compile(
            T_ij[-1][i],
            5*(j+1)
        )

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:41<00:00,  8.35s/it]


TypeError: evaluate() got an unexpected keyword argument 'num_shots'

In [71]:
T_ij

[[<src.preprocessing.make_dataset.SplitWordsDataset at 0x7f82ed452460>,

In [62]:
j_i == len(generated_labels)

True

In [56]:
generated_labels

[['QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'ANSWER',
  'QUESTION',
  'ANSWER',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'ANSWER',
  'ANSWER',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'ANSWER',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'QUESTION',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'QUESTION',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'ANSWER',
  'QUESTION',
  'ANSWER',
  'QUESTION',
  'ANSWER',
  'ANSWER',


In [49]:
b = [train_datas]

In [50]:
b.append([1,2])

In [52]:
b[-1]

[1, 2]

In [15]:
generated_labels ,_, _  = trainers[1].evaluate(
            train_datas[1], 
            models[0][0],
            n_shots = 10,
            return_generated_dataset=True
        )

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:47<00:00,  4.73s/it]


In [16]:
len(generated_labels)

10

In [17]:
T_n= deepcopy(train_datas)

In [25]:
T_n[0][0]

[{'pattern': "In the phrase 'R&D . Suggestion. Date', the R&D is in the header, in a question, or in an answer? [MASK]",
  'label': 'NONE'},
 {'pattern': "In the phrase 'R&D . Suggestion. Date', the Suggestion. is in the header, in a question, or in an answer? [MASK]",
  'label': 'QUESTION'},
 {'pattern': "In the phrase 'R&D . Suggestion. Date', the Date is in the header, in a question, or in an answer? [MASK]",
  'label': 'QUESTION'},
 {'pattern': "In the phrase ' Licensee Yes No 597005708 R&D QUALITY IMPROVEMENT SUGGESTION/ SOLUTION FORM Name / Phone Ext. . M', the Licensee is in the header, in a question, or in an answer? [MASK]",
  'label': 'ANSWER'},
 {'pattern': "In the phrase ' Licensee Yes No 597005708 R&D QUALITY IMPROVEMENT SUGGESTION/ SOLUTION FORM Name / Phone Ext. . M', the Yes is in the header, in a question, or in an answer? [MASK]",
  'label': 'QUESTION'},
 {'pattern': "In the phrase ' Licensee Yes No 597005708 R&D QUALITY IMPROVEMENT SUGGESTION/ SOLUTION FORM Name / Ph

In [27]:
for i in range(len(T_n[0])):
    for j in range(len(T_n[0][i])):
        if i < len(generated_labels):
            T_n[0][i][j]['label'] = generated_labels[i][j]

In [28]:
T_n

In [75]:
generated_labels[0]

['QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'ANSWER',
 'QUESTION',
 'ANSWER',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'ANSWER',
 'ANSWER',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'ANSWER',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'QUESTION',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'QUESTION',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'QUESTION',
 'ANSWER',
 'QUESTION',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'ANSWER',
 'QUESTION',
 'QUESTIO

In [52]:
train_datas[0]